In [27]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import imgaug.augmenters as ia
from sklearn.datasets import make_moons
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt



In [28]:
image_directory = "/content/drive/MyDrive/capstone/all_img"
label_directory = "/content/drive/MyDrive/capstone/all_label"

image_files = [f for f in os.listdir(image_directory) if f.endswith((".jpg", ".png", ".jpeg"))]

label_files = [os.path.splitext(f)[0] + ".txt" for f in image_files]

image_paths = [os.path.join(image_directory, f) for f in image_files]
label_paths = [os.path.join(label_directory, f) for f in label_files]


In [29]:
def parse_label(label_path):
    with open(label_path, 'r') as label_file:
        line = label_file.readline().strip()

    parts = line.split()
    class_id = int(parts[0])
    x_center, y_center, width, height = map(float, parts[1:])

    return class_id, x_center, y_center, width, height


In [30]:

def create_yolo_model(input_shape, num_predictions=5):
    input_layer = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu')(input_layer)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Flatten()(x)
    output_layer = layers.Dense(num_predictions)(x)
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    return model

input_shape = (240, 240, 3)
yolo_model = create_yolo_model(input_shape)
yolo_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

yolo_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 conv2d_9 (Conv2D)           (None, 240, 240, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 120, 120, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 120, 120, 64)      18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 60, 60, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 60, 60, 128)       7385

In [31]:
batch_size = 8
num_epochs = 10


In [32]:
def prepare_dataset(image_paths, label_paths, input_size):
    images = []
    labels = []
    for image_path, label_path in zip(image_paths, label_paths):
        image = cv2.imread(image_path)
        image = cv2.resize(image, input_size)

        images.append(image)
        labels.append(parse_label(label_path))

    return np.array(images), np.array(labels)


In [33]:
images, labels = prepare_dataset(image_paths, label_paths, input_shape[:2])

In [34]:
print("Number of images:", len(images))
print("Number of labels:", len(labels))

Number of images: 2607
Number of labels: 2607


In [35]:
import imgaug.augmenters as iaa

augmentor = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Affine(rotate=(-10, 10)),
    iaa.Multiply((0.7, 1.3), per_channel=0.2),]  )


augmented_images = augmentor.augment_images(images)


In [36]:
def format_labels(labels):
    formatted_labels = []
    for label in labels:
        class_id, x_center, y_center, width, height = label
        formatted_label = [class_id, x_center, y_center, width, height]
        formatted_labels.append(formatted_label)
    return np.array(formatted_labels)

formatted_labels = format_labels(labels)

In [37]:
# Early Stopping
es = EarlyStopping(
    monitor='val_loss',
    patience=200,
    verbose=1,
    mode=min
)

#Checkpoint
mc = ModelCheckpoint(
    'best_m.h5',
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True
)


history = yolo_model.fit(
    augmented_images,formatted_labels,
    batch_size=batch_size,
    epochs=num_epochs,
)

history = model.fit(
    trainX,trainy,
    validation_data=(testX,testy),
    epochs=4000,
    callbacks=[es,mc],
    verbose=2
)


Epoch 1/10
326/326 [==============================] - 6s 14ms/step - loss: 9.5670 - accuracy: 0.3076
Epoch 2/10
326/326 [==============================] - 4s 13ms/step - loss: 9.6122 - accuracy: 0.4745
Epoch 3/10
326/326 [==============================] - 5s 15ms/step - loss: 9.6122 - accuracy: 0.4745
Epoch 4/10
326/326 [==============================] - 4s 14ms/step - loss: 9.6122 - accuracy: 0.4745
Epoch 5/10
326/326 [==============================] - 4s 14ms/step - loss: 9.6122 - accuracy: 0.4745
Epoch 6/10
326/326 [==============================] - 5s 15ms/step - loss: 9.6122 - accuracy: 0.4745
Epoch 7/10
326/326 [==============================] - 4s 14ms/step - loss: 9.6122 - accuracy: 0.4745
Epoch 8/10
326/326 [==============================] - 5s 14ms/step - loss: 9.6122 - accuracy: 0.4745
Epoch 9/10
326/326 [==============================] - 5s 15ms/step - loss: 9.6122 - accuracy: 0.4745
Epoch 10/10
326/326 [==============================] - 5s 14ms/step - loss: 9.6122 - accura

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

plt.savefig('es_loss.png')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.savefig('acc_es.png')
plt.show()